In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet50
import pkbar
from tqdm import tqdm
import json
import dataset_factory
from experiment import Experiment
import coco_dataset
import vocab
import warnings
warnings.filterwarnings("ignore")

f = open("./default.json")

config = json.load(f)

Using CUDA
Using CUDA


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
exp_name = 'default'

print("Running Experiment: ", exp_name)
exp = Experiment(exp_name)
exp.run()
exp.test()

Running Experiment:  default
loading annotations into memory...
Done (t=1.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Using the saved vocab.
loading annotations into memory...
Done (t=0.93s)
creating index...
index created!
loading annotations into memory...
Done (t=0.84s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


  0%|          | 1/235 [00:04<15:53,  4.08s/it]

predicted caption: a plate of food is on a table
actual caption: a table with cups , a plate with cheese and tomato and a platter of bread


  1%|          | 2/235 [00:04<11:19,  2.92s/it]

predicted caption: a cat sitting on a desk next to a keyboard
actual caption: a gray laptop with an apple in the center of the lid sits on a dark surface


...

100%|█████████▉| 234/235 [01:30<00:00,  6.01it/s]

predicted caption: a clock tower is on the side of a building
actual caption: a ballroom type fixture displays a clock which hangs from a <unk> lighting boom over a large boulder
predicted caption: a person riding skis down a snow covered slope
actual caption: person in white and black snow boarding down a hill toward the camera


100%|██████████| 235/235 [01:30<00:00,  2.58it/s]

Test Performance: Loss: 1.5139005681301685, Bleu1: 6.298173363668614, Bleu4: 0.6183268786483774


(1.5139005681301685,
 <function caption_utils.bleu1(reference_captions, predicted_caption)>,
 <function caption_utils.bleu4(reference_captions, predicted_caption)>)